# 4. 実用のためのメソッド

## 4-1. クローラーの特性

- 状態を持つクローラー
- JavaScriptを解釈するクローラー
- 不特定多数のWebサイトを対象とするクローラー

### 4-1-1. 状態を持つクローラー

- HTTP
  - ステートレスに設計されたプロトコル。
- Cookie
  - HTTP上で状態を保持するために利用される。
  - HTTPリクエスト・レスポンスに小さなデータを更かして送受信する仕組み。
  - サーバーがHTTPレスポンスのSet-Cookieヘッダーで値を送信する。
  - クライアントはその値を保持する。
  - クライアントが次回以降、そのWebサイトにHTTPリクエストを送る際は、<br>保存しておいた値をCookieヘッダーで送る。
- Requests.Sessionオブジェクト
  - サーバーから受信したCookieを次回以降のリクエストで自動的に送信できる。
- Referer
  - 1つ前に閲覧したページ（リンク元のページ）のURLをサーバーに送るためのHTTPヘッダー。

### 4-1-2. JavaScriptを解釈するクローラー

- Selenium
- Puppeteer
- Pyppeteer

### 4-1-3. 不特定多数のWebサイトを対象とするクローラー

- 同時並列処理
- 抜き出したデータのストレージ保存
- 保存時の書き込み速度

## 4-2. 収集したデータの利用に関する注意

- 著作権
- 利用規約と個人情報

## 4-3. クロール先の負荷に関する注意

### 4-3-1. 同時接続とクロール間隔

### 4-3-2. robots.txtによるクローラーへの指示

robots.txtの代表的なディレクティブ

|ディレクティブ|説明|
|:-:|:-:|
|User-agent|以降のディレクティブの対象となるクローラーを表す|
|Disallow|クロールを禁止するパス|
|Allow|クロールを許可するパス|
|Sitemap|XMLサイトマップのURLを表す|
|Crawl-delay|クロール間隔を表す|

```txt
<!-- すべてのページをクロールを許可しない -->
User-agent: *
Disallow: /
```

```txt
<!-- すべてのページのクロールを許可する -->
User_agent: *
Disallow:
```

```txt
<!-- 特定のクローラーに対してクロールを許可しない -->
User_agent: *
Disallow: /old/
Disallow: /tmp
```

```txt
<!-- /articles/以下のみクロールを許可する -->
User-agent: *
Allow: /articles/
Disallow: /
```

In [2]:
# robots.txtのパース
import urllib.robotparser

rp = urllib.robotparser.RobotFileParser()
# set_url()で、robots.txtのURLを設定する。
rp.set_url('https://www.python.org/robots.txt')
# read()で、robots.txtを読み込む。
# can_fetch()の第1引数にUser-agentの文字列、第2引数に対象のURLを指定する。
# 指定したURLのクロールが許可されているかどうかを取得できる。
rp.can_fetch('mybot', 'https://python.org/')

False

#### robots meta タグ

```html
<meta name="robots" content="noindex">
```

- nofollow
  - このページ内のリンクをたどることを許可しない。
- noarchive
  - このページをアーカイブとして保存することを許可しない。
- noindex
  - このページを検索エンジンにインデックスすることを許可しない。

### 4-3-3. XMLサイトマップ

### 4-3-4. 連絡先の明示

- GooglebotのUser−Agentヘッダー
  - `Mozilla/5.0 (compatible, Googlebot/2.1; +http://www.google.com/bot.html)`

### 4-3-5. ステータスコードとエラー処理

- HTTP通信におけるエラーの分類
  - ネットワークレベルのエラー
    - DNS解決の失敗
    - 通信のタイムアウト
    - サーバーと正常に通信できていない場合に発生する
  - HTTPレベルのエラー
    - サーバーと正常に通信はできているものの、HTTPレベルで問題がある場合に発生する

#### HTTP通信におけるエラーへの対処法

- 時間をおいてリトライする場合
  - リトライ数が増える度に指数関数的にリトライ間隔を増やす
  - 1秒、2秒、4秒、8秒

#### Pythonによるエラー処理

In [1]:
# ステータスコードに応じたエラー処理
!python error_handling.py

Retrieving http://httpbin.org/status/200,404,503...
Status: 503
Waiting 1 seconds...
Retrieving http://httpbin.org/status/200,404,503...
Status: 404
Error!


In [2]:
# ライブラリtenacityを使ってリトライ処理を簡潔に書く
!python error_handling_with_tenacity.py

Retrieving http://httpbin.org/status/200,404,503...
Status: 503
Retrieving http://httpbin.org/status/200,404,503...
Status: 404
Error!


## 4-4. 繰り返しの実行を前提とした設計

- 更新されたデータだけを取得できるようにするため
- エラーなどで停止した後に途中から再開できるようにするため

&nbsp;

- 更新されたデータのみを取得する方法
- クロール先のWebサイトに変化があったときに検知する方法

### 4-4-1. 更新されたデータだけを取得する

#### HTTPのキャッシュ

- キャッシュに関するHTTPヘッダー
  - Cache-Control
    - コンテンツをキャッシュしてもよいかなど、キャッシュ方針を細かく指示する。
  - Expires
    - コンテンツの有効期限を示す。
  - ETag
    - コンテンツの識別子を表す。
    - コンテンツが変わると、ETagの値も変わる。
  - Last-Modified
    - コンテンツの最終更新日を表す。
  - Pragma
    - Cache_Controlと似たものだったが、現在は後方互換のためだけに残されている。
  - Vary
    - 値に含まれるリクエストヘッダーの値が変わると、サーバーが返すレスポンスも変わることを表す。

&nbsp;

- 強いキャッシュ
  - クライアントは一度レスポンスをキャッシュすると、
  - 有効期限が切れるまではレクエストを送らず、
  - キャッシュされたレスポンスを使う。
  - キャッシュが有効な間はサーバーにリクエストを送らないので、サーバーに負担をかけない。
  - Cache-Control
  - Expires
- 弱いキャッシュ
  - クライアントは一度レスポンスをキャッシュすると、
  - 次回から条件付きのリクエストを送り、
  - サーバーは更新がない場合、`304 Not Modified`というステータスコードで空のレスポンスを返す。
  - このステータスコードが返ってきた場合、
  - クライアントはキャッシュされたレスポンスを使う。
  - サーバーには毎回リクエストを送るものの、レスポンスボディの処理を省略できる。
  - キャッシュがない場合よりは、サーバー負荷を軽減できる。
  - Last-Modified
  - ETag

In [4]:
# Pythonの公式ドキュメントのWebサイトに`curl`コマンドでリクエストを送る
!curl -v https://docs.python.org/3/ > dev/null

/bin/bash: dev/null: No such file or directory


#### PythonでHTTPキャッシュを扱う

- `pip install "CacheControl[filecache]"`

In [5]:
# CacheControlを使ってキャッシュを処理する
# 1回目
# 通常通りサーバーから取得して、`.webcache`ディレクトリ内にキャッシュする。
!python request_with_cache.py

from_cache: False
status_code: 200

<!DOCTYPE html>

<html>
  <head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" /><title>3.11.3 Documentation</title><meta name="viewport" content="width=device-width, initial-scale=1.0">
    
    <link rel="stylesheet" type="text/css" href="_static/pygments.css" />
    <link rel="stylesheet" type="text/css" href="_static/pydoctheme.css?digest=2d3badd06fe70b34b68db01f99471ce1624ffe4a" />
    
    <script data-url_root="./" id="documentation_options" src="_static/documentation_options.js"></script>
    <script src="_static/jquery.js"></script>
    <script src="_static/underscore.js"></script>
    <script src="_static/doctools.js"></script>
    
    <script src="_static/sidebar.js"></script>
    
    <link rel="search" type="application/opensearchdescription+xml"
          title="Search within Python 3.11.3 documentation"
          href="_static/opensearch.xml"/>
    <link rel="author" title="Abou

In [6]:
# CacheControlを使ってキャッシュを処理する
# 2回目
# 対象ページが更新されていない場合、キャッシュされた結果を使用する。
!python request_with_cache.py

from_cache: True
status_code: 200

<!DOCTYPE html>

<html>
  <head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" /><title>3.11.3 Documentation</title><meta name="viewport" content="width=device-width, initial-scale=1.0">
    
    <link rel="stylesheet" type="text/css" href="_static/pygments.css" />
    <link rel="stylesheet" type="text/css" href="_static/pydoctheme.css?digest=2d3badd06fe70b34b68db01f99471ce1624ffe4a" />
    
    <script data-url_root="./" id="documentation_options" src="_static/documentation_options.js"></script>
    <script src="_static/jquery.js"></script>
    <script src="_static/underscore.js"></script>
    <script src="_static/doctools.js"></script>
    
    <script src="_static/sidebar.js"></script>
    
    <link rel="search" type="application/opensearchdescription+xml"
          title="Search within Python 3.11.3 documentation"
          href="_static/opensearch.xml"/>
    <link rel="author" title="About

- 今回は、キャッシュの保存先としてファイルを使用した。
- デフォルトは、メモリ上に保存される。
- key-valueストアのRedisへの保存も可能。
- <https://cachecontrol.readthedocs.io/en/latest>

### 4-4-2. クロール先の変化を検知する

- CSSセレクターやXPathで取得しようとした要素が存在しなくなった。
- 要素は変わらず存在するものの、意図したものとは違う値が得られてしまった。

#### 正規表現でバリデーションする

- re

In [8]:
# 正規表現で価格として正しいかチェックする。
!python validate_with_re.py

Traceback (most recent call last):
  File "/Users/takeru/@LEARNING/Python/python_crawling_and_scraping_gihyo/ch04/validate_with_re.py", line 20, in <module>
    validate_price('無料')
  File "/Users/takeru/@LEARNING/Python/python_crawling_and_scraping_gihyo/ch04/validate_with_re.py", line 14, in validate_price
    raise ValueError(f'Invalid price: {value}')
ValueError: Invalid price: 無料


#### JSON Schemaでバリデーションする

- jsonschema

In [10]:
# jsonschemaによるバリデーション
!python validate_with_jsonschema.py

Traceback (most recent call last):
  File "/Users/takeru/@LEARNING/Python/python_crawling_and_scraping_gihyo/ch04/validate_with_jsonschema.py", line 31, in <module>
    validate({
  File "/Users/takeru/@LEARNING/Python/python_crawling_and_scraping_gihyo/venv/lib/python3.10/site-packages/jsonschema/validators.py", line 1121, in validate
    raise error
jsonschema.exceptions.ValidationError: '無料' does not match '^[0-9,]+$'

Failed validating 'pattern' in schema['properties']['price']:
    {'pattern': '^[0-9,]+$', 'type': 'string'}

On instance['price']:
    '無料'
